In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import pandas as pd


In [4]:
#dfstackreset_indexpivotdf
df= pd.DataFrame([[0, 1], [2, 3]],index=['cat', 'dog'],columns=['weight', 'height'])

print('{:*^80}'.format('dataframe'))
df

print('{:*^80}'.format('stacked dataframe'))
df.stack()

print('{:*^80}'.format('stacked dataframe with index in column'))
df.stack().reset_index(level = 1)   #AttributeError: 'Series' object has no attribute 'pivot_table' , so must convert to dataframe before pivot_table
stacked = df.stack().reset_index(level = 1)  

print('{:*^80}'.format('pivot_table recovered original dataframe (with extra name for columns)'))
recovered_df1 = df.stack().reset_index(level = 1).pivot_table(index = stacked.index, columns = 'level_1',values = 0) #pivot_table orders columns alphabetically,specifying values parameter prevents creation of useless multiindex column 
recovered_df1.columns.name = None  #remove 'level_1' column.name
recovered_df1


***********************************dataframe************************************


,weight,height
cat,0,1
dog,2,3


*******************************stacked dataframe********************************


cat  weight    0
     height    1
dog  weight    2
     height    3
dtype: int64

*********************stacked dataframe with index in column*********************


,level_1,0
cat,weight,0
cat,height,1
dog,weight,2
dog,height,3


*****pivot_table recovered original dataframe (with extra name for columns)*****


,height,weight
cat,1.0,0.0
dog,3.0,2.0


In [7]:
#dfstackdfpivotdf
print('{:*^80}'.format('dataframe'))
df
print('{:*^80}'.format('stack and convert to dataframe to expose pivot_table'))
stacked_df = pd.DataFrame(df.stack())
stacked_df

print('{:*^80}'.format('rather than unstack, pivot_table achieves the same'))
idx_lvl0, idx_lvl1 = stacked_df.index.get_level_values(0), stacked_df.index.get_level_values(1)
recovered_df2 = stacked_df.pivot_table(index=idx_lvl0,columns = idx_lvl1,values = 0)
recovered_df2


***********************************dataframe************************************


,weight,height
cat,0,1
dog,2,3


**************stack and convert to dataframe to expose pivot_table**************


0
cat weight  0
    height  1
dog weight  2
    height  3

***************rather than unstack, pivot_table achieves the same***************


,height,weight
cat,1.0,0.0
dog,3.0,2.0


In [8]:
#dfmeltadd indexpivotdf
print('{:*^80}'.format('dataframe'))
df
print('{:*^80}'.format('melting loses index information'))
melted = df.melt()   #melt appends columns into new "variable" column, while stack adds columns to new inner index layer (same information end up different places)
melted

print('{:*^80}'.format('manually enrich index')) # until this is solved: https://github.com/pandas-dev/pandas/issues/17440
melted.index = ['cat','dog']*2   #list(df.index)*len(df.columns) for more generalizable index generation
melted                           

print('{:*^80}'.format('pivot_table recovered original dataframe (with extra name for columns)'))
recovered_df3 = melted.pivot_table(index = melted.index, columns = 'variable',values = 'value')
recovered_df3.columns.name=None  #remove 'variable' column.name
recovered_df3

#melting loses index while pivot_table requires index parameter


***********************************dataframe************************************


,weight,height
cat,0,1
dog,2,3


************************melting loses index information*************************


,variable,value
0,weight,0
1,weight,2
2,height,1
3,height,3


*****************************manually enrich index******************************


,variable,value
cat,weight,0
dog,weight,2
cat,height,1
dog,height,3


*****pivot_table recovered original dataframe (with extra name for columns)*****


,height,weight
cat,1.0,0.0
dog,3.0,2.0
